In [23]:
"""
Calculate fire severity statistics within AFD observations from MODIS and VIIRS.
Severity index: Composite Burn Severity (CBI), calculated in GEE via Parks (2018)

Author: maxwell.cook@colorado.edu
"""

import ee
import geemap

import os, time
import pandas as pd
import geopandas as gpd

import warnings
warnings.filterwarnings("ignore") # suppresses annoying geopandas warning

ee.Authenticate()

ee.Initialize(project='jfsp-aspen')

proj = 'EPSG:5070'

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Success")

Success


In [2]:
def monitor_export(task, sleep=30):
    """ Monitors EE export task """
    while task.active():
        print('Waiting for export to finish..\n\tPatience young padawan.')
        time.sleep(sleep)  # Check every 30 seconds
    
    # Get the status of the task
    status = task.status()
    
    # Check if the task failed or succeeded
    if status['state'] == 'COMPLETED':
        print("Export completed successfully !!!!")
    elif status['state'] == 'FAILED':
        print(f"Export failed! Bummer. Reason: {status.get('error_message', 'Unknown error')}")
    else:
        print(f"Export ended with state: {status['state']}")

print("Functions ready!")

Functions ready!


In [3]:
# Load the FRP observations
afd = ee.FeatureCollection('projects/jfsp-aspen/assets/AFD/combined-afd_aspen-fires_2018_to_2023')
print(afd.first().propertyNames().getInfo())

['fid', 'FRP', 'NIFC_NAME', 'LONGITUDE', 'ACQ_YEAR', 'ACQ_TIME', 'SCAN', 'BRIGHTNESS', 'CONFIDENCE', 'TYPE', 'SATELLITE', 'na_l3name', 'DAYNIGHT', 'TRACK', 'INSTRUMENT', 'VID', 'NIFC_ID', 'START_YEAR', 'ACQ_DATETI', 'ACQ_MONTH', 'WF_CESSATI', 'VERSION', 'BRIGHT_T31', 'DISCOVERY_', 'afdID', 'LATITUDE', 'system:index', 'ACQ_DATE']


In [4]:
afd = afd.select(['afdID'])
print(afd.first().propertyNames().getInfo())

['system:index', 'afdID']


In [5]:
# Load the CBI mosaic
cbi = ee.Image('projects/jfsp-aspen/assets/CBI/afd_aspen-fires_cbi_mosaic')
print(cbi.bandNames().getInfo())

['CBI', 'CBI_bc', 'rbr']


In [6]:
# Set up the reductions for CBI statistics in AFD observations and export.

In [10]:
# Multiple reducer
mean_stdDev = ee.Reducer.mean().combine(
    reducer2=ee.Reducer.stdDev(), sharedInputs=True).combine(
    reducer2=ee.Reducer.percentile([90]), sharedInputs=True)

# Perform the reduction by AFD observations
cbi_stats = cbi.reduceRegions(
    collection=afd,
    reducer=mean_stdDev, # Mean and Standard Deviation CBI, CBI_bc, rbr
    scale=30
)
print("Submitted !")

Submitted !


In [11]:
# Export table to Drive.

cbi_stats = cbi_stats.map(lambda feature: feature.setGeometry(None)) # drop geometry column

export_task = ee.batch.Export.table.toDrive(
    collection=cbi_stats,
    description='afd_aspen-fires_cbi-stats',
    fileFormat='CSV', 
    fileNamePrefix='afd_aspen-fires_cbi-stats',
    folder='CBI'
)

# Start the export task
export_task.start()
print("Export to Earth Engine Asset started!")
# Monitor the task until it's finished
monitor_export(export_task, 120) # print every 2 min

Export to Earth Engine Asset started!
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Export completed successfully !!!!


In [17]:
# Load the exported table.
fp = os.path.join(projdir,'data/earth-engine/exports/cbi/afd_aspen-fires_cbi-stats.csv')
cbi = pd.read_csv(fp)
cbi.columns

Index(['system:index', 'CBI_bc_mean', 'CBI_bc_p90', 'CBI_bc_stdDev',
       'CBI_mean', 'CBI_p90', 'CBI_stdDev', 'afdID', 'rbr_mean', 'rbr_p90',
       'rbr_stdDev', '.geo'],
      dtype='object')

In [18]:
cbi.drop(columns=['system:index','.geo'], inplace=True)
cbi.head()

,CBI_bc_mean,CBI_bc_p90,CBI_bc_stdDev,CBI_mean,CBI_p90,CBI_stdDev,afdID,rbr_mean,rbr_p90,rbr_stdDev
0,1.345921,2.475556,0.766035,1.405602,2.335556,0.621546,MODIS12004,176.382395,404.000000,135.908069
1,0.639773,2.446000,0.921366,0.811641,2.306000,0.770013,MODIS12037,87.950558,329.046809,136.633111
2,2.357810,2.945151,0.702763,2.240195,2.740000,0.587089,MODIS12038,385.084445,569.364006,169.026412
3,1.987865,2.790000,0.773277,1.931922,2.600000,0.633484,MODIS12039,293.339556,473.598030,145.768924
4,1.765142,2.820000,0.872010,1.751206,2.634022,0.715643,MODIS12040,246.728583,477.625000,156.294200


In [22]:
# Join back to the afd data.
fp = os.path.join(projdir,'data/spatial/mod/AFD/combined-afd_aspen-fires_2018_to_2023_buffer.gpkg')
afd = gpd.read_file(fp)
afd.columns

/opt/miniconda3/envs/aspen-fire/lib/python3.10/site-packages/geopandas/io/file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
/opt/miniconda3/envs/aspen-fire/lib/python3.10/site-packages/geopandas/io/file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
/opt/miniconda3/envs/aspen-fire/lib/python3.10/site-packages/geopandas/io/file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
/opt/miniconda3/envs/aspen-fire/lib/python3.10/site-packages/geopandas/io/file.py:399: FutureWarning: erro

Index(['LATITUDE', 'LONGITUDE', 'BRIGHTNESS', 'SCAN', 'TRACK', 'ACQ_DATE',
       'ACQ_TIME', 'SATELLITE', 'INSTRUMENT', 'CONFIDENCE', 'VERSION',
       'BRIGHT_T31', 'FRP', 'DAYNIGHT', 'TYPE', 'VID', 'NIFC_ID', 'NIFC_NAME',
       'START_YEAR', 'DISCOVERY_DATE', 'WF_CESSATION_DATE', 'na_l3name',
       'ACQ_MONTH', 'ACQ_YEAR', 'ACQ_DATETIME', 'afdID', 'geometry'],
      dtype='object')